In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger, \
    LambdaCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import numpy as np
import tensorflow as tf
import json
from sklearn.utils.class_weight import compute_class_weight
from dataset.ImageDatasetGenerator import ImageDatasetGenerator
from dataset.generate_dataset import GenerateDataset
from pathlib import Path
import math

In [2]:
def calculate_steps(dataset_size, batch_size):
    return math.ceil(dataset_size / batch_size)

In [3]:
guyana_dataset_path = '/datadrive/notebooks/downloaded_data/full_data_top_23_0.1_resize_672_672'

with open('/datadrive/bdfr.metadata') as json_file:
        guyana_dataset_metadata = json.load(json_file)
        
generate_dataset_guyana = GenerateDataset(guyana_dataset_path, guyana_dataset_metadata)

generate_dataset_guyana.load_dataset('None')

dataset_guyana = generate_dataset_guyana.all_dataset()

In [4]:
train_data_guyana = dataset_guyana['train']
val_data_guyana = dataset_guyana['val']
test_data_guyana = dataset_guyana['test']

In [5]:
batch_size = 64
output_dim = (224, 224)

In [6]:
train_image_generator = ImageDatasetGenerator(train_data_guyana['files'], train_data_guyana['labels'], batch_size, output_dim,
                                        guyana_dataset_metadata, train_mode=True, random_crop=True)
val_image_generator = ImageDatasetGenerator(val_data_guyana['files'], val_data_guyana['labels'], batch_size, output_dim,
                                        guyana_dataset_metadata, train_mode=False, random_crop=False)

In [7]:
len(train_image_generator), len(val_image_generator)

(5233, 1740)

In [8]:
n_classes = generate_dataset_guyana.num_classes()

In [3]:
#np.bincount(train_flow.classes)

array([294, 123, 202, 179, 191, 471, 282, 476, 659, 286, 256, 692, 121,
       376, 130, 131, 159, 145, 161, 190, 123, 143, 139], dtype=int64)

In [4]:
'''
class_weights = {i: weight for i, weight in enumerate(compute_class_weight('balanced', np.unique(train_flow.classes), train_flow.classes))}
n_classes = len(class_weights)
class_weights, n_classes
'''

C:\Users\calvom\Miniconda3\envs\tensorflow_env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22], y=[ 0  0  0 ... 22 22 22] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


({0: 0.8768115942028986,
  1: 2.0957935666313183,
  2: 1.2761515281962978,
  3: 1.4401263055623026,
  4: 1.349647165945823,
  5: 0.5473091479737838,
  6: 0.914122725871107,
  7: 0.5415601023017903,
  8: 0.39117239559279543,
  9: 0.9013377926421404,
  10: 1.0069633152173914,
  11: 0.3725182206584569,
  12: 2.130434782608696,
  13: 0.6855920444033302,
  14: 1.982943143812709,
  15: 1.9678061732492533,
  16: 1.6212742685261143,
  17: 1.7778110944527736,
  18: 1.6011342155009451,
  19: 1.3567505720823798,
  20: 2.0957935666313183,
  21: 1.8026755852842808,
  22: 1.8545511416953393},
 23)

In [9]:

'''
img_generator = image_dataset_from_directory('C:\\Users\\calvom\\ThesisMaster\\notebooks\\downloaded_data\\training_top_23_0.1_resize_672_672', 
                                             batch_size=32, 
                                             image_size=(224, 224), 
                                             shuffle=True, 
                                             label_mode='categorical')
'''

base_model = ResNet50(weights='imagenet', include_top=False)

input_img = Input((224, 224,3))

x = preprocess_input(input_img)

encoded_features = base_model(x)

x3 = GlobalAveragePooling2D()(encoded_features)
x4 = Dense(1024, activation='relu')(x3)
x5 = Dropout(0.4)(x4)

predictions = Dense(n_classes, activation='softmax', name='softmax', kernel_regularizer=l2(0.01),
                         bias_regularizer=l2(0.01), kernel_initializer=tf.keras.initializers.glorot_normal())(x5)

# this is the model we will train
model = Model(inputs=input_img, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional resnet50 layers
for layer in base_model.layers:
    layer.trainable = True

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192  

In [10]:

callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-12,
                          verbose=1),
        EarlyStopping(monitor='val_loss', patience=10, verbose=1)]
    

op = Adam(lr=1e-4, decay=1e-4)
    
model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy', 'top_k_categorical_accuracy'])


model.fit(train_image_generator, 
                    steps_per_epoch=calculate_steps(len(train_image_generator), batch_size),
                    epochs=100,
                    validation_steps=calculate_steps(len(val_image_generator), batch_size),
                    validation_data=val_image_generator, use_multiprocessing=True, workers=16, callbacks=callbacks)
#model.fit(train_flow, epochs=10, class_weight=class_weights)
model.save('/datadrive/bdfr_models/step_1_model')

C:\Users\calvom\Miniconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
82/82 [==============================] - 824s 10s/step - loss: 3.0855 - accuracy: 0.2080 - val_loss: 5.0790 - val_accuracy: 0.1425
Epoch 2/10
82/82 [==============================] - 836s 10s/step - loss: 2.5177 - accuracy: 0.3114 - val_loss: 5.6305 - val_accuracy: 0.1305
Epoch 3/10
82/82 [==============================] - 808s 10s/step - loss: 2.2701 - accuracy: 0.3562 - val_loss: 4.0128 - val_accuracy: 0.2293
Epoch 4/10
82/82 [==============================] - 833s 10s/step - loss: 2.1339 - accuracy: 0.3960 - val_loss: 3.6548 - val_accuracy: 0.2552
Epoch 5/10
82/82 [==============================] - 903s 11s/step - loss: 1.9643 - accuracy: 0.4331 - val_loss: 3.8092 - val_accuracy: 0.2080
Epoch 6/10
82/82 [==============================] - 906s 11s/step - loss: 1.8951 - accuracy: 0.4467 - val_loss: 3.4749 - val_accuracy: 0.2172
Epoch 7/10
82/82 [==============================] - 808s 10s/step - loss: 1.8284 - accuracy: 0.4500 - val_loss: 4.2179 - val_accuracy: 0.2213
Epoch 

In [13]:
'''
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
'''

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [14]:
'''
for layer in base_model.layers[:143]:
    layer.trainable = False
for layer in base_model.layers[143:]:
    layer.trainable = True
'''

In [15]:
'''
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
'''

In [16]:
'''
#model.fit(train_flow, epochs=10, validation_data=validation_flow, class_weight=class_weights)
model.fit(train_image_generator, 
                    steps_per_epoch=calculate_steps(len(train_image_generator), batch_size),
                    epochs=20,
                    validation_steps=calculate_steps(len(val_image_generator), batch_size),
                    validation_data=val_image_generator, multiprocessing=True, workers=16)

model.save('/datadrive/bdfr_models/step_2_model')
'''

Epoch 1/10
82/82 [==============================] - 1130s 14s/step - loss: 1.9536 - accuracy: 0.4470 - val_loss: 2.5939 - val_accuracy: 0.2931
Epoch 2/10
82/82 [==============================] - 1108s 14s/step - loss: 1.6003 - accuracy: 0.5305 - val_loss: 2.5880 - val_accuracy: 0.3115
Epoch 3/10
82/82 [==============================] - 1105s 14s/step - loss: 1.4428 - accuracy: 0.5663 - val_loss: 2.5352 - val_accuracy: 0.3247
Epoch 4/10
82/82 [==============================] - 1150s 14s/step - loss: 1.3244 - accuracy: 0.5896 - val_loss: 2.7713 - val_accuracy: 0.3155
Epoch 5/10
82/82 [==============================] - 1188s 14s/step - loss: 1.2705 - accuracy: 0.6149 - val_loss: 2.5658 - val_accuracy: 0.3253
Epoch 6/10
82/82 [==============================] - 1055s 13s/step - loss: 1.1517 - accuracy: 0.6506 - val_loss: 2.9589 - val_accuracy: 0.2868
Epoch 7/10
82/82 [==============================] - 1086s 13s/step - loss: 1.1485 - accuracy: 0.6409 - val_loss: 2.8133 - val_accuracy: 0.3098